In [2]:
###################################
## My best attempt at a GP in 2d ##
###################################
import numpy as np
import matplotlib.pyplot as plt
from fvgp import GP
import time
%load_ext autoreload
%autoreload 2
from itertools import product
x_pred1D = np.linspace(0,1,1000).reshape(-1,1)

Here we will use the Goldstein-Price Function which is defined as:
$$
f(x) = [1 + (x_1 + x_2 + 1)^2 (19-14x_1 + 3x_1^2 - 14x_2 + 6x_1x_2 + 3x_2^2)] \times [30 + (2x_1 - 3x_2)^2(18 - 32x_1 + 12x_1^2 + 48x_2 - 36x_1x_2 + 27x_2^2)]
$$

In [1]:
def goldstein_price_function(x1,x2):
    return (1 + (x1 + x2 + 1)**2 * (19- 14*x1 + 3*x2 -14*x2 + 6*x1*x2 + 3*x2**2) ) * ( 30 + (2*x1 - 3*x2)**2 * ( 18 - 32*x1 + 12*x1**2 + 48*x2 - 36*x1*x2 + 27*x2**2) )

In [1]:
import numpy as np 
import plotly.graph_objects as go
from gpcam.gp_optimizer import GPOptimizer

# Plotting the results on a 3D plot
def plot(x,y,z,data = None):
    fig = go.Figure()
    fig.add_trace(go.Surface(x = x, y = y, z=z))
    if data is not None:
        fig.add_trace(go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2],
                                   mode='markers',marker=dict(size=12,
                                                              color=data[:,2],         # Set color equal to a variable
                                                              #colorscale='Viridis',   # Choose a colorscale
                                                              opacity=0.8)
                                   )
        )

    fig.update_layout(title='Plot', autosize=True,
                  width=800, height=800,
                  margin=dict(l=65, r=50, b=65, t=90),scene=dict(xaxis_title='Temperature', yaxis_title='Cycle', zaxis_title='Energy'))


    fig.show()

In [2]:
# Define the temperature range and the step size
temp = np.arange(0, 100, 1)

# Define the cycle range and the step size
cycle = np.arange(0, 600, 1)

# Create a mesh grid of the temperature and cycle values
T, C = np.meshgrid(temp, cycle)

# Define the energy as a function of temperature and cycle

energy = -C**2 - 4*C*(np.max(T) -T) + 600000

#Plot the results
plot(T,C,energy)

### Collecting Noisy Data at Specific Temperatures

In [3]:
# Creating arrays of data with temperatures 10, 50, and 80 and cycles 1 to 600
# Create the initial 2D array with 10 in the first column and values from 1 to 100 in the second column
array_10 = np.column_stack((np.full((100,), 10), np.arange(1, 601,6)))
array_50 = np.column_stack((np.full((100,), 50), np.arange(1, 601,6)))
array_80 = np.column_stack((np.full((100,), 80), np.arange(1, 601,6)))

# Stack the arrays vertically to create the final array
x_data = np.vstack((array_10, array_50, array_80))


print(x_data.shape)

temp = x_data[:,0]
cyc = x_data[:,1]

y_data = -cyc**2 - 4*cyc*(np.max(temp) -temp) + 600000

# Add normally distributed random errors to the second column
error = np.random.normal(loc=100000, scale=10000, size=(300,))
y_data = y_data + error

print(y_data.shape)


(300, 2)
(300,)


### Defining the Design Space




In [4]:
# Define the grid size
n = 100

# Design Space Limits
c_low = 0
c_high = 800
temp_low = 1
temp_high = 300

# Create a design space
y_space = np.linspace(c_low,c_high,n)
x_space = np.linspace(temp_low,temp_high,n)
x_space,y_space = np.meshgrid(x_space,y_space)

# Reshape the arrays into a 2-column array with 10000 rows
my_space = np.vstack((x_space.reshape(-1), y_space.reshape(-1))).T

### Specifying the GP Model

In [5]:
def get_distance_matrix(x1,x2):
    d = np.zeros((len(x1),len(x2)))
    for i in range(x1.shape[1]):
        d += (x1[:,i].reshape(-1, 1) - x2[:,i])**2
    return np.sqrt(d + 1e-16)

def s(x, slope, offset):
    o = slope * x + offset
    return o

def kernel(x1,x2,hps,obj):
    d = get_distance_matrix(x1,x2)
    noise = 0.0
    #print(len(x1),len(x2))
    if len(x1) == len(x2):
        noise = np.identity(len(x1)) * np.outer(s(x1,hps[2],hps[3]),s(x2,hps[2],hps[3]))
    #print(np.min(noise),np.max(noise))
    k = hps[0] * obj.squared_exponential_kernel(d,hps[1]) + noise
    return k


# Mean Function
def mean(x, hps,obj):
    y = -(x[:,1]/hps[3])**2 - hps[4]*x[:,1]*(hps[5] - x[:,0]) + hps[6]
    return y




In [10]:

# Initializing the GP Model
my_gpo = GPOptimizer([2],np.array([temp_low,temp_high],[c_low,c_high]))                                     # I had to change 2 to [2] and change np.array([[]]) to np.array([])
my_gpo.init_gp(np.array([0.01, 0.01, 0.01, 1, 1, 80, 400000]), gp_mean_function=mean)

#my_gpo.init_gp(np.array([1.50218860e+01, 2.50038865e+02, 5.08911155e-02, 4.88959850e-01,
#        5.52568959e+01]), gp_kernel_function=kernel, gp_mean_function=mean)

#print("tell")
my_gpo.tell(x_data,y_data,variances=0.001*y_data)

my_gpo.train_gp(np.array([[0.001,100],[0.001,100],[0.001,100],[0.1,10],[0.1,10],[30,150],[300000,700000]]), max_iter = 20)

#print("hps: ", my_gpo.hyperparameters)
f = my_gpo.posterior_mean(my_space)["f(x)"]
v = my_gpo.posterior_covariance(my_space)["v(x)"]


f_re = f.reshape(100,100)

plot(x_space,y_space,f_re, data = np.column_stack([my_gpo.x_data,my_gpo.y_data]))


TypeError: Field elements must be 2- or 3-tuples, got '0'

In [ ]:
plot(x_space,y_space,f_re, data = np.column_stack([my_gpo.x_data,my_gpo.y_data]))
